<a href="https://colab.research.google.com/github/castorini/anserini-notebooks-afirm2020/blob/master/afirm2020_rerank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Re-ranking with BERT

We are going to run a [BERT](https://arxiv.org/abs/1810.04805)-based re-ranker over our candidate MS MARCO passages.
BERT is a massively pretrained langauge model that has been shown to improve many downstream NLP tasks.

In this activity, we are going to first learn how to re-rank with BERT and then compare the retrieval effectiveness to BM25.
For more details, please see [this paper](https://arxiv.org/abs/1901.04085).

This notebook is based on the [dl4marco-bert](https://github.com/nyu-dl/dl4marco-bert) repo.

## Setup

Firstly, we need to set up Colab TPU running environment, verify a TPU device is succesfully connected and upload credentials to TPU for GCS bucket usage.
You are going to need a Google account for this step.


In [0]:
import json
import os
import pprint
import time

import tensorflow as tf


assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.


TPU address is grpc://10.48.95.106:8470
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 11868873848394123751),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 620339734624852149),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 9903249103132971663),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 7047032228759654293),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 6513372630341192950),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/de

Secondly, clone the `dl4marco-bert` repo:

In [0]:
import sys

!test -d bert_repo || git clone https://github.com/nyu-dl/dl4marco-bert dl4marco-bert
if not 'dl4marco-bert' in sys.path:
  sys.path += ['dl4marco-bert']

Cloning into 'dl4marco-bert'...
remote: Enumerating objects: 154, done.
remote: Total 154 (delta 0), reused 0 (delta 0), pack-reused 154
Receiving objects: 100% (154/154), 90.00 KiB | 1.87 MiB/s, done.
Resolving deltas: 100% (84/84), done.


## Data

The TensorFlow implementation for passage re-ranking expects the data in TFRecord format.
Since generating these files from the original collection takes 1-2 days, we have already made them available on GCS.

- `BERT_MODEL`: Uncased Base or Large model (we recommend starting with Base as it takes less time to run)
- `OUTPUT_DIR`: The output path in the GCS filesystem to store model checkpoints and evaluation results
- `DATA_DIR`: The data path in the GCS filesystem (contains the TFRecord)

In [0]:
# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
BERT_MODEL = 'uncased_L-12_H-768_A-12' #@param {type:"string"}
BERT_PRETRAINED_DIR = 'gs://cloud-tpu-checkpoints/bert/' + BERT_MODEL
print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))
!gsutil ls $BERT_PRETRAINED_DIR

OUTPUT_DIR = 'gs://afirm2020/bert-output' #@param {type:"string"}
assert OUTPUT_DIR, 'Must specify an existing GCS bucket name'
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

# Now we need to specify the input data dir. Should contain the .tfrecord files 
# and the supporting query-docids mapping files.
DATA_DIR = 'gs://afirm2020/tfrecord' #@param {type:"string"}
print('***** Data directory: {} *****'.format(DATA_DIR))


***** BERT pretrained directory: gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12 *****
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_config.json
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.index
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.meta
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/checkpoint
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/vocab.txt


KeyboardInterrupt: ignored

## Training / Evaluation

With all our data and models in place, we can now train BERT for passage re-ranking and evaluate on the dev set.

Make sure to set `DO_TRAIN` to run training and `DO_EVAL` to evaluate.

In [0]:
# Parameters
USE_TPU = True
DO_TRAIN = True
DO_EVAL = False
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 32
LEARNING_RATE = 1e-6
NUM_TRAIN_STEPS = 400000
NUM_WARMUP_STEPS = 40000
MAX_SEQ_LENGTH = 512
SAVE_CHECKPOINTS_STEPS = 1000
ITERATIONS_PER_LOOP = 1000
NUM_TPU_CORES = 8
BERT_CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
MSMARCO_OUTPUT = True  # Write the predictions to a MS-MARCO-formatted file.
MAX_EVAL_EXAMPLES = None  # Maximum number of examples to be evaluated.
NUM_EVAL_DOCS = 1000  # Number of docs per query in the dev and eval files.
METRICS_MAP = ['MAP', 'RPrec', 'NDCG', 'MRR', 'MRR@10']

In [0]:
import time
import numpy as np
import tensorflow as tf

import metrics
import modeling
import optimization

def create_model(bert_config, is_training, input_ids, input_mask, segment_ids,
                 labels, num_labels, use_one_hot_embeddings):
  """Creates a classification model."""
  model = modeling.BertModel(
      config=bert_config,
      is_training=is_training,
      input_ids=input_ids,
      input_mask=input_mask,
      token_type_ids=segment_ids,
      use_one_hot_embeddings=use_one_hot_embeddings)

  output_layer = model.get_pooled_output()
  hidden_size = output_layer.shape[-1].value

  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):
    if is_training:
      # I.e., 0.1 dropout
      output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)

    return (loss, per_example_loss, log_probs)


def model_fn_builder(bert_config, num_labels, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    tf.logging.info("*** Features ***")
    for name in sorted(features.keys()):
      tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_training = (mode == tf.estimator.ModeKeys.TRAIN)
    (total_loss, per_example_loss, log_probs) = create_model(
        bert_config, is_training, input_ids, input_mask, segment_ids, label_ids,
        num_labels, use_one_hot_embeddings)

    tvars = tf.trainable_variables()

    scaffold_fn = None
    initialized_variable_names = []
    if init_checkpoint:
      (assignment_map, initialized_variable_names
      ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
      if use_tpu:
        def tpu_scaffold():
          tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
          return tf.train.Scaffold()

        scaffold_fn = tpu_scaffold
      else:
        tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

    tf.logging.info("**** Trainable Variables ****")
    for var in tvars:
      init_string = ""
      if var.name in initialized_variable_names:
        init_string = ", *INIT_FROM_CKPT*"
      tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                      init_string)

    output_spec = None
    if mode == tf.estimator.ModeKeys.TRAIN:

      train_op = optimization.create_optimizer(
          total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          train_op=train_op,
          scaffold_fn=scaffold_fn)

    elif mode == tf.estimator.ModeKeys.PREDICT:
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          predictions={
              "log_probs": log_probs,
              "label_ids": label_ids,
          },
          scaffold_fn=scaffold_fn)

    else:
      raise ValueError(
          "Only TRAIN and PREDICT modes are supported: %s" % (mode))

    return output_spec

  return model_fn


def input_fn_builder(dataset_path, seq_length, is_training,
                     max_eval_examples=None):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  def input_fn(params):
    """The actual input function."""

    batch_size = params["batch_size"]
    output_buffer_size = batch_size * 1000

    def extract_fn(data_record):
      features = {
          "query_ids": tf.FixedLenSequenceFeature(
              [], tf.int64, allow_missing=True),
          "doc_ids": tf.FixedLenSequenceFeature(
              [], tf.int64, allow_missing=True),
          "label": tf.FixedLenFeature([], tf.int64),
      }
      sample = tf.parse_single_example(data_record, features)

      query_ids = tf.cast(sample["query_ids"], tf.int32)
      doc_ids = tf.cast(sample["doc_ids"], tf.int32)
      label_ids = tf.cast(sample["label"], tf.int32)
      input_ids = tf.concat((query_ids, doc_ids), 0)

      query_segment_id = tf.zeros_like(query_ids)
      doc_segment_id = tf.ones_like(doc_ids)
      segment_ids = tf.concat((query_segment_id, doc_segment_id), 0)

      input_mask = tf.ones_like(input_ids)

      features = {
          "input_ids": input_ids,
          "segment_ids": segment_ids,
          "input_mask": input_mask,
          "label_ids": label_ids,
      }
      return features

    dataset = tf.data.TFRecordDataset([dataset_path])
    dataset = dataset.map(
        extract_fn, num_parallel_calls=4).prefetch(output_buffer_size)

    if is_training:
      dataset = dataset.repeat()
      dataset = dataset.shuffle(buffer_size=1000)
    else:
      if max_eval_examples:
        # Use at most this number of examples (debugging only).
        dataset = dataset.take(max_eval_examples)
        # pass

    dataset = dataset.padded_batch(
        batch_size=batch_size,
        padded_shapes={
            "input_ids": [seq_length],
            "segment_ids": [seq_length],
            "input_mask": [seq_length],
            "label_ids": [],
        },
        padding_values={
            "input_ids": 0,
            "segment_ids": 0,
            "input_mask": 0,
            "label_ids": 0,
        },
        drop_remainder=True)

    return dataset
  return input_fn


def main(_):
  tf.logging.set_verbosity(tf.logging.INFO)
  
  if not DO_TRAIN and not DO_EVAL:
    raise ValueError("At least one of `DO_TRAIN` or `DO_EVAL` must be True.")

  bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG_FILE)

  if MAX_SEQ_LENGTH > bert_config.max_position_embeddings:
    raise ValueError(
        "Cannot use sequence length %d because the BERT model "
        "was only trained up to sequence length %d" %
        (MAX_SEQ_LENGTH, bert_config.max_position_embeddings))

  tpu_cluster_resolver = None
  if USE_TPU:
    tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(
        TPU_ADDRESS)

  is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
  run_config = tf.contrib.tpu.RunConfig(
      cluster=tpu_cluster_resolver,
      model_dir=OUTPUT_DIR,
      save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
      tpu_config=tf.contrib.tpu.TPUConfig(
          iterations_per_loop=ITERATIONS_PER_LOOP,
          num_shards=NUM_TPU_CORES,
          per_host_input_for_training=is_per_host))

  model_fn = model_fn_builder(
      bert_config=bert_config,
      num_labels=2,
      init_checkpoint=INIT_CHECKPOINT,
      learning_rate=LEARNING_RATE,
      num_train_steps=NUM_TRAIN_STEPS,
      num_warmup_steps=NUM_WARMUP_STEPS,
      use_tpu=USE_TPU,
      use_one_hot_embeddings=USE_TPU)

  # If TPU is not available, this will fall back to normal Estimator on CPU
  # or GPU.
  estimator = tf.contrib.tpu.TPUEstimator(
      use_tpu=USE_TPU,
      model_fn=model_fn,
      config=run_config,
      train_batch_size=TRAIN_BATCH_SIZE,
      eval_batch_size=EVAL_BATCH_SIZE,
      predict_batch_size=EVAL_BATCH_SIZE)

  if DO_TRAIN:
    tf.logging.info("***** Running training *****")
    tf.logging.info("  Batch size = %d", TRAIN_BATCH_SIZE)
    tf.logging.info("  Num steps = %d", NUM_TRAIN_STEPS)
    train_input_fn = input_fn_builder(
        dataset_path=DATA_DIR + "/dataset_train.tf",
        seq_length=MAX_SEQ_LENGTH,
        is_training=True)
    estimator.train(input_fn=train_input_fn,
                    max_steps=NUM_TRAIN_STEPS)
    tf.logging.info("Done Training!")

  if DO_EVAL:
    for set_name in ["dev", "eval"]:
      tf.logging.info("***** Running evaluation *****")
      tf.logging.info("  Batch size = %d", EVAL_BATCH_SIZE)
      max_eval_examples = None
      if MAX_EVAL_EXAMPLES:
        max_eval_examples = MAX_EVAL_EXAMPLES * NUM_EVAL_DOCS

      eval_input_fn = input_fn_builder(
          dataset_path=DATA_DIR + "/dataset_" + set_name + ".tf",
          seq_length=MAX_SEQ_LENGTH,
          is_training=False,
          max_eval_examples=max_eval_examples)

      if MSMARCO_OUTPUT:
        msmarco_file = tf.gfile.Open(
            OUTPUT_DIR + "/msmarco_predictions_" + set_name + ".tsv", "w")
        query_docids_map = []
        with tf.gfile.Open(
            DATA_DIR + "/query_doc_ids_" + set_name + ".txt") as ref_file:
          for line in ref_file:
            query_docids_map.append(line.strip().split("\t"))

      # ***IMPORTANT NOTE***
      # The logging output produced by the feed queues during evaluation is very 
      # large (~14M lines for the dev set), which causes the tab to crash if you
      # don't have enough memory on your local machine. We suppress this 
      # frequent logging by setting the verbosity to WARN during the evaluation
      # phase.
      tf.logging.set_verbosity(tf.logging.WARN)
      
      result = estimator.predict(input_fn=eval_input_fn,
                                 yield_single_examples=True)
      start_time = time.time()
      results = []
      all_metrics = np.zeros(len(METRICS_MAP))
      example_idx = 0
      total_count = 0
      for item in result:
        results.append((item["log_probs"], item["label_ids"]))

        if len(results) == NUM_EVAL_DOCS:

          log_probs, labels = zip(*results)
          log_probs = np.stack(log_probs).reshape(-1, 2)
          labels = np.stack(labels)

          scores = log_probs[:, 1]
          pred_docs = scores.argsort()[::-1]
          gt = set(list(np.where(labels > 0)[0]))

          all_metrics += metrics.metrics(
              gt=gt, pred=pred_docs, metrics_map=METRICS_MAP)

          if MSMARCO_OUTPUT:
            start_idx = example_idx * NUM_EVAL_DOCS
            end_idx = (example_idx + 1) * NUM_EVAL_DOCS
            query_ids, doc_ids = zip(*query_docids_map[start_idx:end_idx])
            assert len(set(query_ids)) == 1, "Query ids must be all the same."
            query_id = query_ids[0]
            rank = 1
            for doc_idx in pred_docs:
              doc_id = doc_ids[doc_idx]
              # Skip fake docs, as they are only used to ensure that each query
              # has 1000 docs.
              if doc_id != "00000000":
                msmarco_file.write(
                    "\t".join((query_id, doc_id, str(rank))) + "\n")
                rank += 1

          example_idx += 1
          results = []

        total_count += 1

        if total_count % 10000 == 0:
          tf.logging.warn("Read {} examples in {} secs. Metrics so far:".format(
              total_count, int(time.time() - start_time)))
          tf.logging.warn("  ".join(METRICS_MAP))
          tf.logging.warn(all_metrics / example_idx)

      # Once the feed queues are finished, we can set the verbosity back to 
      # INFO.
      tf.logging.set_verbosity(tf.logging.INFO)
      
      if MSMARCO_OUTPUT:
        msmarco_file.close()

      all_metrics /= example_idx

      tf.logging.info("Eval {}:".format(set_name))
      tf.logging.info("  ".join(METRICS_MAP))
      tf.logging.info(all_metrics)


if __name__ == "__main__":
  tf.app.run()


W1130 14:14:02.535755 140405660108672 module_wrapper.py:139] From dl4marco-bert/modeling.py:92: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



W1130 14:14:03.694923 140405660108672 estimator.py:1994] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7fb249c3f620>) includes params argument, but params are not passed to Estimator.


INFO:tensorflow:Using config: {'_model_dir': 'gs://dl4marco-trimmed/first_sent_output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.95.40.186:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb24aee4160>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.95.40.186:8470', '_evaluation_master': 'grpc://10.95.40.186:8470', '_is_chief': True, '_num_ps_

I1130 14:14:03.699721 140405660108672 estimator.py:212] Using config: {'_model_dir': 'gs://dl4marco-trimmed/first_sent_output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.95.40.186:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb24aee4160>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.95.40.186:8470', '_evaluation_master': 'grpc://10.95.40

INFO:tensorflow:_TPUContext: eval_on_tpu True


...

I1130 23:41:26.103616 140405660108672 error_handling.py:101] training_loop marked as finished


INFO:tensorflow:Done Training!


I1130 23:41:26.105116 140405660108672 <ipython-input-4-5e2a1ac8dfcb>:276] Done Training!


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


*TODO: Visualizations to compare BM25 and BERT runs?*